Sai Saradha Kalidaikurichi Lakshmanan
EECS 531 - Assignment A1

Exercise 1:
Make a notebook that explains and implements a blurring filter. Refer to slide 3 of
lecture 3b. For this exercise, you should implement the filter “by hand”, and not call a library
function.

Filtering primarily involves performing an operation on a neighborhood of pixels for image enhancement. This operation creates a new value for the pixel and that value is at the coordinate position equal to the center of the neighborhood. There are two broad types of filtering based on the domain they are done in - spatial filtering and frequency filtering. In this exercise, the blur filters used are those that operate in the spatial domain. There are further divisions depending on the kernel types, such as separable or non-separable, linear or non-linear, etc., Filtering is used in a wide range of applications while the most common in image processing is to use this technique in preprocessing steps mainly for blurring the image or sharpening the image. Blurring is nothing but smoothing the image and is equivalent to a low pass filter, avoids high frequency/intensity components and as a result, blurs the edges also to some extent. In this exercise, we implement few simple blurring filters and study their effects on a set of images (stored in the 'Test_Images' folder).

The list of filters implemented in this exercise are - 
Box/Average, Weighted Average, Gaussian, Bilinear, Median, Bilateral, Max and Min blur filters.

The following study has been done (more details of the below topics in the corresponding sections):
1. Testing with and without padding, duplication and mirroring
2. Linear and NonLinear filters
3. Speed of filtering - runtime analysis
4. Effect on Color vs. Grayscale images
5. Varying size of kernels
6. Salt Pepper Noise reduction by Non-linear filters

In [6]:
# Code Section begins:
import numpy as np
import os
import cv2
from scipy import ndimage
import scipy
import time

In [ ]:
class blur_filters:
    
    def __init__(self):
        pass
    
    def _pad(self, Img, fs):
        # pad with zeros on all sides depending on the filter size
        ps = int(fs/2)
        pimg = np.pad(Img, (ps,), 'constant', constant_values = 0)
        return pimg
    
    def kernel_img_conv(self, Img, fs, kernelb):
        # get padded image:
        pimg = self._pad(Img, fs)
        # get the image shape
        height = Img.shape[0]
        wd = Img.shape[1]
        # create an array for the blurred image
        Img_final = np.zeros((height, wd))
        for h in range(height):
            for w in range(wd):
                Img_final[h,w] =(np.sum(np.multiply(kernelb, pimg[h:h+fs,w:w+fs])))
        return Img_final
                                         
    def box(self, Img, fs):
        # Kernel coefficient
        k2 = fs**2
        # The main Kernel matrix (but this is not needed, since we are only going to find the average)
        kernelb = np.ones((fs, fs))
        # get padded image:
        pimg = self._pad(Img, fs)
        # get the image shape
        height = Img.shape[0]
        wd = Img.shape[1]
        # create an array for the blurred image
        Img_final = np.zeros((height, wd))
        for h in range(height):
            for w in range(wd):
                Img_final[h,w] = (1/k2)*(np.sum(pimg[h:h+fs,w:w+fs]))
        # Img2 = (((Img_final - Img_final.min()) / (Img_final.max() - Img_final.min())) * 255.9).astype(np.uint8)
        # Img2 = exposure.equalize_adapthist(Img_final/np.max(np.abs(Img_final)), clip_limit=0.03)
        Img2 = Img_final.astype(np.uint8)
        cv2.namedWindow('Box Blur')
        cv2.imshow('Box Blur',Img2)
        
        '''# now try convolve (inbuilt operation):
        Img3 = ndimage.convolve(Img, (1/k2)*kernelb, mode = 'constant', cval=0.0)
        cv2.namedWindow('Blurred Conv')
        cv2.imshow('Blurred Conv', Img3)
        # As we can see, both return same results'''
        cv2.waitKey(0)
        return Img2
    
    def weighted_avg(self, Img, fs):
        # Kernel coefficient
        k2 = (fs**2)+1
        # The main Kernel matrix (but this is not needed, since we are only going to find the average)
        # the kernel is weighted in the centre, for example [[1 1 1][1 2 1][1 1 1]], the value 2 can be changed to any other 
        # larger value
        kernelb = np.ones((fs, fs))
        kernelb[int(fs/2)][int(fs/2)] += 1
        Img_final = self.kernel_img_conv(Img, fs, (1/k2)*kernelb)
        Img2 = Img_final.astype(np.uint8)
        cv2.namedWindow('Weighted Avg')
        cv2.imshow('Weighted Avg',Img2)
        cv2.waitKey(0)
        return Img2
    
    def gaussian(self, Img, fs):
        gridx, gridy = scipy.mgrid[-(int(fs/2)):int(fs/2)+1, -(int(fs/2)):int(fs/2)+1]
        sigma = 1
        # nc = 1/(2*np.pi*(sigma**2))
        gkernel = np.exp(-(gridx**2 + gridy**2)/(2*(sigma**2))).astype(np.float32)
        nc = 1/(gkernel.sum()).astype(np.float32)
        gkernel = nc*gkernel
        Img_final = self.kernel_img_conv(Img, fs, gkernel)
        Img2 = Img_final.astype(np.uint8)
        cv2.namedWindow('Gaussian Blur')
        cv2.imshow('Gaussian Blur',Img2)
        cv2.waitKey(0)

        # interesting experiment:
        # Inspired from CMU lecture, we can introduce shadow effect in images by
        # overlaying the original image and the gaussian blurred image
        self.shadow_effect_analysis(Img, Img2)
        return Img2
    
    def bilinear(self, Img, fs):
        # attempted only for filter of size 3
        kernelb = [[1/16,2/16,1/16],[2/16,4/16,2/16],[1/16,2/16,1/16]]
        Img_final = self.kernel_img_conv(Img, 3, kernelb)
        Img2 = Img_final.astype(np.uint8)
        cv2.namedWindow('Bilinear blur')
        cv2.waitKey(0)
        return Img2
    
    def medianf(self, Img, fs):
        # value of the pixel is equal to the median of the neighborhood
        # get padded image:
        pimg = self._pad(Img, fs)
        # get the image shape
        height = Img.shape[0]
        wd = Img.shape[1]
        # create an array for the blurred image
        Img_final = np.zeros((height, wd))
        for h in range(height):
            for w in range(wd):
                Img_final[h,w] = np.median(pimg[h:h+fs,w:w+fs])
        Img2 = Img_final.astype(np.uint8)
        cv2.namedWindow('Median Blur')
        cv2.imshow('Median Blur',Img2)
        cv2.waitKey(0);
        return Img2

    def salt_pepper(self, Img, fs):
        height = Img.shape[0]
        wd = Img.shape[1]
        Img3 = np.zeros((height, wd))
        cv2.randu(Img3, 0, 255);
        i_bl = Img3 < 30;
        i_wh = Img3 > 225;
        sp_img = Img;
        sp_img[i_wh] = 255;
        sp_img[i_bl] = 0;
        cv2.namedWindow('SaltPepper Noise');
        cv2.imshow('SaltPepper Noise', sp_img);
        cv2.waitKey(0);
        Img_median = self.medianf(sp_img, fs);
        return Img_median
    
    def maxf(self, Img, fs):
        # value of the pixel is equal to the median of the neighborhood
        # get padded image:
        pimg = self._pad(Img, fs)
        # get the image shape
        height = Img.shape[0]
        wd = Img.shape[1]
        # create an array for the blurred image
        Img_final = np.zeros((height, wd))
        for h in range(height):
            for w in range(wd):
                Img_final[h,w] = np.amax(pimg[h:h+fs,w:w+fs])
        Img2 = Img_final.astype(np.uint8)
        cv2.namedWindow('Max Filter')
        cv2.imshow('Max Filter',Img2)
        return Img2
    
    def minf(self, Img, fs):
        # value of the pixel is equal to the median of the neighborhood
        # get padded image:
        pimg = self._pad(Img, fs)
        # get the image shape
        height = Img.shape[0]
        wd = Img.shape[1]
        # create an array for the blurred image
        Img_final = np.zeros((height, wd))
        for h in range(height):
            for w in range(wd):
                Img_final[h,w] = np.amin(pimg[h:h+fs,w:w+fs])
        Img2 = Img_final.astype(np.uint8)
        cv2.namedWindow('Min Filter')
        cv2.imshow('Min Filter',Img2)
        return Img2

    def transparentOverlay(src , overlay , pos=(0,0),scale = 1):
        return

    def separable_show(self, Img, fs):
        # Kernel coefficient
        k2 = fs
        # The main Kernel matrix (but this is not needed, since we are only going to find the average)
        kernelb = np.ones(fs)
        # get padded image:
        pimg = self._pad(Img, fs)
        # get the image shape
        height = Img.shape[0]
        wd = Img.shape[1]
        # create an array for the blurred image
        Img_final = np.zeros((height, wd))
        for h in range(height):
            for w in range(wd):
                Img_final[h,w] = (1/k2)*(np.sum(pimg[h,w:w+fs]))
        pimg = self._pad(Img_final, fs)
        Img_final = np.zeros((height,wd))
        for w in range(wd):
            for h in range(height):
                Img_final[h,w] = (1/k2)*(np.sum(pimg[h:h+fs,w]))
        Img2 = Img_final.astype(np.uint8)
        cv2.namedWindow('Box Blur Separable')
        cv2.imshow('Box Blur Separable',Img2)
        cv2.waitKey(0)
        return

    def color_image_process(self, Img):
        rim, gim, bim = np.split(Img, 3, axis=2)
        aim = np.ones_like(rim) * 255
        splits = [rim, gim, bim], aim
        return splits

    def shadow_effect_analysis(self, Img, Img2):
        '''result = self.transparentOverlay(Img,Img2)
        cv2.namedWindow("Result",cv2.WINDOW_NORMAL)
        cv2.imshow("Result",result)
        cv2.waitKey(0)'''
        return
    
    def read_image(self, directory, filename):
        if filename.startswith("I") and (filename.endswith(".jpg") or filename.endswith(".bmp") or filename.endswith(".png") or filename.endswith(".gif")):
            print(filename)
            img = cv2.imread(directory + filename,-1)
            if img is None:
                print('Invalid image:' + filename)
                return None
            else:
                cv2.imshow('Image', img)
                cv2.waitKey(0)
                return img
            
    def main(self, img_folder, filter_choice, filter_size):
        directory = os.fsencode(img_folder)
        options = {'box':self.box,
                   'weighted_avg':self.weighted_avg,
                   'gaussian': self.gaussian,
                   'bilinear': self.bilinear,
                   'medianf': self.medianf,
                   'maxf':self.maxf,
                   'minf':self.minf,
            }
        for file in os.listdir(directory):
            tic = time.time()
            filename = os.fsdecode(file)
            Img = self.read_image(img_folder, filename)
            if Img is None:
                continue
            if len(Img.shape) ==3:
                print('Entering color image routine')
                splits, aim = self.color_image_process(Img)
                i=0
                Img_f = []
                Img_sp = []
                for im in splits:
                    im = im[:,:,0].copy().astype(np.uint8)
                    temp_img =options[filter_choice](im, int(filter_size))
                    # temp_img = ((((temp_img - temp_img.min()) / (temp_img.max() - temp_img.min())) * 255.9).astype(np.uint8))
                    Img_f.append(temp_img)
                    if filter_choice == 'medianf':
                        # to study the effectiveness of median filter, we
                        # introduce salt-pepper noise and see how it works
                        temp_sp = self.salt_pepper(im, int(filter_size))
                        Img_sp.append(temp_sp)
                Img_final = np.dstack((Img_f[0], Img_f[1], Img_f[2], aim)).copy()
                Img_final = Img_final.astype(np.uint8)
                cv2.namedWindow('Blurred Image')
                cv2.imshow('Blurred Image', Img_final)
                cv2.imwrite('E1_images\\'+str(filename),Img_final)
                if filter_choice == 'medianf':
                    Img_final = np.dstack((Img_sp[0], Img_sp[1], Img_sp[2], aim)).copy()
                    Img_final = Img_final.astype(np.uint8)
                    cv2.namedWindow('SPN removed Image')
                    cv2.imshow('SPN removed Image', Img_final)
            else:
                print('Entering grayscale routine')
                Img_final = options[filter_choice](Img, int(filter_size))
                Img_final = Img_final.astype(np.uint8)
                cv2.imwrite('E1_images\\'+str(filename),Img_final)
                if filter_choice == 'medianf':
                    # to study the effectiveness of median filter, we
                    # introduce salt-pepper noise and see how it works
                    self.salt_pepper(Img, int(filter_size))
                toc = time.time() - tic
                print("Running time without separating the filters: ", str(toc))
                
                print("Now running time for separable filters: ")
                tic = time.time()
                self.separable_show(Img, int(filter_size))
                toc = time.time()- tic
                print(str(toc))
            #break
        return

In [ ]:
if __name__ == "__main__":

    # creating an instance of the blur filters class
    filter_class = blur_filters()

    # Call to the main function
    # Arguments to the main function are directory to test images
    # filter choice and filter size

    tic = time.time()
    filter_class.main("..\\Test_Images\\",input("Enter the filter choice: "), input("Enter the filter size: "))
    toc = time.time() - tic
    print("Running time: " + str(toc))

Analysis :
As we can see from the resutls, these blurring/smoothing filters result in reducing the false edges and hence most of the time used before edge detection. The general procedure is to pad the original image with sufficient zeros (equal to fs/2 on all sides) and then convolve the image with the kernel to replace the values of pixel in the image. The general implementation of filter can be given as (from Gonzalez),

$$g(x,y) = \frac{\sum_{s=-a}^{a}\sum_{t=-b}^{b}w(s,t)f(x+s,y+t)}{\sum_{s=-a}^{a}\sum_{t=-b}^{b}w(s,t)}$$

Box Filter :
Box filter is an average filter. Every pixel in the image is replaced by the average of its neighborhood and this naturally means the coefficients of the kernel are all equal. They also act as low pass filters. A kernel of size fs (assuming odd and square kernels), has coefficients equal to 1/m^2. In this implementation, kernel filters have been implemented by one simple optimization -

    Img_final[h,w] = (1/fs^2)(np.sum(pimg[h:h+fs,w:w+fs]))
    
Here instead of convolving with the kernel, I have directly summed up the coefficients (multiplication removed) and this makes it much faster than the other filters. This is possible because of all kernel elements being equal to 1 (or rather 1/m^2)
This is can be expressed as,

Weighted Average - Same as the average except that centre pixels (or some pixels in general) are given more importance than the others. The other pixels in the kernel are weighted inversely of their distance from the centre. 

Gaussian - The kernel matrix for this filter are sampled from a 2D gaussian function (refer implementation) and the function can be given as,

$$f(i,j) = \frac{1}{2\pi \sigma^{2}}e^{-\frac{(i^{2}+j^{2})}{2\sigma^{2}}}$$


The weights in the matrix fall of as the distance from center pixel increases. This is a linear separable filter (explanation in the following section).

Median, Max, Min - These are noon linear filters based on ordering the pixels by some metric, that is, replacing the pixel in the image by median or the max or the min value of the neighborhood (neighborhood decided by the filter size). Median filters represent the 50th percentile of the ranked pixels and are very useful to remove the salt pepper noise (results in the next section) 
Median filter can be given as,

$$\hat{f}(x,y) = \underset{(s,t)\in S_{xy}}{\mathrm{median}} \{g(s,t)\}$$

where s and t are defined by the neighborhood
Max filters are used to find the brightest points in the image and represent the 100th percentile results and can be expressed as 

$$\hat{f}(x,y) = \underset{(s,t)\in S_{xy}}{\mathrm{max}} \{g(s,t)\}$$

Min filters are used for finding the darkest points in the image and all the three filters are used to reduce pepper noise in the image.

$$\hat{f}(x,y) = \underset{(s,t)\in S_{xy}}{\mathrm{min}} \{g(s,t)\}$$


1. Output of different filters (all of them fixed at size 5) -
Order of images :
1. Box filter   2. Gaussian Filter  3. Weighted Avg filter  4. Median Filter  5. Max Filter  6. Min Filter

![Alt text](imgs/I1_box.png?raw=true "I1_box")
![Alt text](imgs/I1_Gaussian.png?raw=true "I1_maxf")
![Alt text](imgs/I1_wa.png?raw=true "I1_wa")
![Alt text](imgs/I1_Median_blurd.png?raw=true "I1_Median_blurd")
![Alt text](imgs/I1_maxf.png?raw=true "I1_maxf")
![Alt text](imgs/I1_minf.png?raw=true "I1_minf")



Filters of varying size: 
As we would expect, filters of large size blur the image more than the filters of smaller size. Here is the comparison of average filters of size 9 and 15.
![Alt text](imgs/I1_box_9.bmp?raw=true "I1_box_9")
![Alt text](imgs/_box_15.bmp?raw=true "_box_15")

Color vs. grayscale images:
I have implemented blurring filters for both color and grayscale images. Color images take a bit longer because the process has to happen in each of the channels (R, G and B)
Here is an example of generating blurred R, G and B images
![Alt text](imgs/I1_R.png?raw=true "I1_R")
![Alt text](imgs/I1_G.png?raw=true "I1_G")
![Alt text](imgs/I1_B.png?raw=true "I1_B")

Separable Kernels :
As discussed in class, some filters, such as the box, weighted average, bilinear and gaussian are all separable. As a result,  2D convolutions can be done as 2 separate 1D operations, as shown below :

$$\begin{bmatrix} 1 & 1 & 1 \\ 1 & 1 & 1 \\ 1 & 1 & 1 \end{bmatrix} = \left[ \begin{array}{c} 1 \\ 1 \\ 1 \end{array} \right] \times \left[ \begin{array}{cc} 1 & 1 & 1 \end{array} \right]$$

The cost of convolution with a non-separable filter is $fs^{2}\times fs^{2}$, while that of the separable kernels is $2fs\times fs^{2}$. I did a simple runtime analysis to verify that this optimizes the operations. 
![Alt text](imgs/separable_runtime_analysis.PNG?raw=true "separable_runtime_analysis")
An observation is that this runtime difference is significant for filters of large sizes (as seen in example, the filter is of size 31).![Alt text](imgs/box_blur_separabel.png?raw=true "box_blur_separabel")

Salt Pepper noise reduction by Non-linear filters:
In this exercise, I have introduced Salt Pepper noise, this noise is the white and black dots on the image. 
Below is the grayscale image of SPN
![Alt text](imgs/SPN.PNG?raw=true "SPN")
and this is the result after applying the median filter :
![Alt text](imgs/I1_SPN_removed.PNG?raw=true "I1_SPN_removed")